목표 : 비상장 외감기업 결측치 채우기
=====================
- inf는 max로 대체, -inf는 min으로 대체
- 정규성 검사
- 정규성 -> fillna(mean 평균)
- 정규성X -> fillna(median 중앙값)

import modules

In [285]:
import pandas as pd
import numpy as np

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

from scipy.stats import shapiro, anderson, kstest, norm

현재 디렉토리: /Users/haph/Study/ubion/Project2/비상장/2.데이터전처리


환경설정

In [286]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

load file

In [287]:
unlisted = pd.read_csv("../1.데이터셋/Train_Test/unlisted_train_data.csv")
# 영업년수와 상장년수가 있더라

In [288]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업투자현금비율', '영업재무현금비율', '도입기',
       '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [289]:
len(unlisted)

14844

# describe

In [290]:
unlisted

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
0,(주)투씨에스지,66390,2015,-8.540,-26.820,-7.400,-2.940,114.400,106.740,649.860,613.270,27.610,11.780,13.920,93.560,13.340,52.410,4.900,2.200,4.730,3.590,1.610,-0.060,1.507,-20.610,-20.610,-21.010,-14.320,29.670,-21.120,3879.670,60.850,2806.980,24.140,9.100,18.540,2.520,3.540,2.450,102.540,2.230,2.230,36.130,2.380,16.000,0.983,-0.099,0.104,0,0,0,1,1.000
1,(주)지앤이바이오텍,78322,2017,48.440,152.580,24.980,25.600,193.160,184.270,106.440,75.250,16.370,33.960,39.280,70.410,48.440,61.030,28.730,20.590,36.300,36.130,25.890,0.420,-0.062,151.055,151.050,315.100,29.390,120.610,19.350,2852.690,35.920,138.720,36.820,55.920,76.570,0.976,16.720,2.400,4.100,1.400,1.400,3.720,2.230,10.000,0.259,-0.887,5.127,0,0,1,0,0.000
2,철인종합건설(주),82687,2018,16.290,46.630,9.030,6.880,180.110,132.960,124.370,118.070,19.370,42.150,3.950,94.780,44.570,16.340,7.100,5.130,9.810,8.390,6.150,-0.630,-0.053,8.001,8.000,33.980,-76.100,-24.830,137.090,2573.740,12.130,6703.310,9.240,55.520,70.220,1.313,5.540,1.870,1020.400,1.380,1.360,9.700,1.590,25.000,0.061,-0.063,0.072,0,0,1,0,0.000
3,(주)지오엠씨,57636,2018,-10.980,-35.720,-9.580,-35.010,113.290,95.160,686.170,611.470,26.820,10.340,0.240,88.120,12.720,-67.160,-13.020,-45.840,-11.670,-11.670,-43.560,8.580,0.135,-4.098,-4.100,0.140,-27.000,-11.150,-41.160,5.430,0.080,5.690,0.290,-15730.770,-6455.050,0.274,0.410,0.390,14.450,0.280,0.270,1.940,0.310,38.000,0.702,-7.594,-1.362,0,0,0,1,1.000
4,주식회사새롬디앤씨,82915,2018,249.940,2343.090,93.850,35.460,228.750,193.340,60.120,52.250,4.010,42.010,0.130,74.640,62.450,163.120,40.530,21.200,53.800,53.640,28.360,-0.050,9.994,-44.387,-44.390,-56.650,231.860,101.920,178.880,5974.490,78.420,441.260,29.630,71.550,94.550,2.647,184.140,7.090,21.920,1.910,1.890,16.040,2.140,9.000,0.153,-1.182,-5.008,0,1,0,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14839,아시아드파크주식회사,90895,2020,17.250,31.350,21.420,29.310,125.340,124.220,0.000,0.000,68.330,10.460,1.370,51.710,0.000,0.000,-13.480,-19.230,-5.590,-7.650,-11.190,1.230,-1.609,-12.185,-12.180,8.820,-27.230,-33.150,-31.330,3776.900,35.700,81.250,48.850,-39.370,70.250,0.731,28.910,1.490,1.350,0.700,0.680,1.280,1.470,6.000,0.942,-2.740,-1.464,0,1,0,0,1.000
14840,(주)참미르,69511,2016,539.050,0.000,31.350,319.910,4738.120,4738.120,6.170,0.280,0.000,12.240,130.630,12.500,94.180,3.570,3.180,33.720,3.890,3.740,39.660,-1.590,-0.410,-7.243,-7.240,-26.410,-3.660,-86.740,-3.660,846.450,7.560,8.640,77.170,43.700,91.830,0.098,335.760,0.660,0.110,0.090,0.090,0.110,0.660,12.000,0.525,-0.776,4.006,0,0,0,1,0.000
14841,(주)이천설봉온천,85340,2022,-1.200,-1.250,-1.200,-9.320,4.080,4.080,0.000,0.000,95.890,-12.960,0.380,0.550,0.000,0.000,-0.470,-3.630,2.420,2.410,18.770,26.420,22.941,-0.216,-0.220,296.060,-0.630,67.920,-0.630,1613.950,9.710,9.770,75.570,-4.810,46.790,0.129,129.240,37.260,0.130,0.130,0.130,0.130,37.260,12.000,0.803,-2.186,-1.986,0,1,0,0,1.000
14842,(주)홈포인트,87381,2019,22.470,25.350,18.690,31.570,716.380,153.580,493.710,74.240,73.700,77.080,62.210,89.580,16.840,8.320,1.300,2.270,3.640,3.490,6.390,6.390,0.376,-14.245,-14.250,-15.350,-3.390,-11.790,-5.520,1111.320,10.900,198.100,18.410,12.340,56.830,0.592,90.580,3.920,10.450,0.570,0.550,5.580,0.610,10.000,0.159,-2.907,-1.363,0,1,0,0,0.000


In [291]:
unlisted.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14639.000,14844.000,14844.000,14844.000,14844.000,14844.000
mean,67803.271,2019.848,17.069,3607.268,3.299,-20.537,396.521,323.104,1369.995,966.185,41.329,2.501,91.008,45.677,38.901,13.681,1.811,-4.219,2.716,2.311,-13.507,8.353,-3.800,6.156,6.872,42.738,268.213,173.467,196133.771,6759.214,30.727,409857.795,40.657,-83.077,10.913,0.942,339.045,4.323,8337.509,1.131,1.013,29.415,2.852,21.418,0.443,-160.160,-141.390,0.223,0.363,0.196,0.218,0.368
std,23620.609,2.819,1252.761,245521.865,23.459,457.262,5278.556,5155.300,25422.528,21814.645,29.801,41.605,2442.869,28.582,25.864,734.703,24.351,258.003,12.328,12.300,165.409,57.095,492.483,53.746,55.425,911.386,10684.049,6769.542,21236977.635,355672.205,1686.359,16668086.473,642.157,2230.082,1040.130,1.285,30988.135,7.827,369881.882,1.387,1.268,859.925,5.168,11.227,1.638,13636.282,16348.832,0.416,0.481,0.397,0.413,0.482
min,10037.000,2014.000,-86467.650,-940370.860,-277.190,-21110.050,0.070,0.000,0.000,0.000,0.000,-1830.280,-3.490,0.010,0.000,-4261.270,-949.400,-8281.590,-309.770,-309.800,-8429.040,-1554.580,-59524.677,-100.000,-97.670,-99.900,-100.000,-99.910,-100.000,-68167.780,-229.460,-123276675.000,-3280.110,-184890.760,-94627.640,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1368867.400,-1947057.000,0.000,0.000,0.000,0.000,0.000
25%,52473.000,2018.000,-4.412,-5.200,-2.880,-4.690,56.040,31.355,62.712,38.343,17.557,-19.683,1.610,22.435,18.517,-4.527,-2.450,-4.820,-1.080,-1.490,-3.290,0.130,-0.135,-4.314,-5.285,-13.812,-5.080,-12.110,-5.600,968.762,4.978,9.598,8.030,0.000,23.727,0.181,4.067,1.510,0.690,0.350,0.300,0.510,1.100,13.000,0.085,-2.497,-1.756,0.000,0.000,0.000,0.000,0.000
50%,70903.500,2022.000,2.860,1.570,1.830,2.380,107.865,71.805,164.550,105.935,41.830,2.870,8.490,43.350,35.265,3.735,1.405,1.610,2.550,2.220,2.810,1.330,-0.008,0.673,1.975,4.195,-0.090,3.750,-1.080,2295.045,11.150,29.095,16.025,11.590,56.000,0.670,7.240,2.950,2.090,0.830,0.750,1.480,2.010,20.000,0.161,-0.779,-0.759,0.000,0.000,0.000,0.000,0.000
75%,87057.750,2022.000,14.400,17.475,7.360,10.400,197.290,148.572,373.355,243.058,61.170,26.730,31.977,67.857,57.142,14.280,6.400,6.950,6.820,6.400,9.100,4.340,0.046,11.303,12.935,25.355,12.610,22.273,8.082,4868.297,19.320,98.303,32.135,35.662,74.763,1.249,13.550,4.973,7.123,1.450,1.310,3.800,3.350,27.000,0.385,-0.057,0.222,0.000,1.000,0.000,0.000,1.000
max,101635.000,2022.000,113999.840,27854555.600,1108.560,21579.540,447760.110,447760.110,2238929.900,1793120.750,1109.290,99.490,275285.960,100.000,99.980,81898.380,1026.650,8884.250,110.740,109.710,2136.190,2702.500,5597.416,5081.217,5081.220,97496.850,1154781.210,720869.030,2580453089.000,43327598.520,205392.780,1618245220.000,70947.820,8226.360,125.590,36.583,3767342.720,344.060,36941363.190,43.960,43.960,93448.220,344.060,113.000,85.287,188183.733,227051.800,1.000,1.000,1.000,1.000,1.000


In [292]:
unlisted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14844 entries, 0 to 14843
Data columns (total 53 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                14844 non-null  object 
 1   거래소코드              14844 non-null  int64  
 2   회계년도               14844 non-null  int64  
 3   CASH FLOW 대 부채비율   14844 non-null  float64
 4   CASH FLOW 대 차입금비율  14844 non-null  float64
 5   CASH FLOW 대 총자본비율  14844 non-null  float64
 6   CASH FLOW 대 매출액비율  14844 non-null  float64
 7   유동비율               14844 non-null  float64
 8   당좌비율               14844 non-null  float64
 9   부채비율               14844 non-null  float64
 10  유동부채비율             14844 non-null  float64
 11  차입금의존도             14844 non-null  float64
 12  순운전자본비율            14844 non-null  float64
 13  현금비율               14844 non-null  float64
 14  유동자산구성비율           14844 non-null  float64
 15  자기자본구성비율           14844 non-null  float64
 16  자기자본순이익률           148

In [293]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업투자현금비율', '영업재무현금비율', '도입기',
       '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [294]:
unlisted.isna().sum().sort_values(ascending=False)
# 결측치는 뒤에서 정규성 검사를 하고 대체한다.

영업재무현금비율             205
회사명                    0
유형자산회전율                0
유형자산증가율                0
부가가치                   0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
경영자본회전률                0
비유동자산증가율               0
총자본회전률                 0
비유동자산회전률               0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
매출액증가율                 0
유동자산증가율                0
거래소코드                  0
총자본증가율                 0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0
순운전자본비율                0


In [295]:
unlisted.drop(['부실판단', '도입기', '성장기', '성숙기', '쇠퇴기'], axis=1).isin([0]).sum().sort_values(ascending=False)

자본분배율                1477
이윤분배율                1337
CROE                 1294
총자산회전율               1293
총자산증가율               1292
차입금의존도               1221
CASH FLOW 대 차입금비율    1212
매출채권회전률              1031
자기자본순이익률              691
자기자본구성비율              531
유동부채비율                530
부채비율                  530
유형자산증가율               240
매출액증가율                160
설비투자효율                125
유형자산회전율                87
금융비용부담률                72
현금비율                   68
비유동자산증가율               30
총자본회전률                 30
경영자본회전률                16
영업년수                   16
비유동자산회전률               13
총자본영업이익률                8
총자본증가율                  8
쭈피처                     8
CASH FLOW 대 매출액비율       7
유동자산회전률                 7
경영자본순이익률                7
총자본사업이익률                6
CASH FLOW 대 총자본비율       5
매출액영업이익률                5
매출액순이익률                 4
순운전자본비율                 3
CASH FLOW 대 부채비율        3
유동자산증가율                 2
총자본투자효율                 1
당좌비율                    1
영업투자현금비율    

In [296]:
unlisted.isin([np.inf]).sum().sort_values(ascending=False)
# inf가 하나 있다. 뒤에서 max 값으로 대체한다.

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [297]:
unlisted.isin([-np.inf]).sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

# 범주 분할 1현금흐름 / 2안정성 / 2수익성 / 3유동성 / 4성장성 / 5생산성 / 6활동성 / 7시장지표 및 기타

In [298]:
# 비상장 all data는 unlisted
# 비상장 label들

labels = unlisted[
    [
        '거래소코드',
        '회계년도',
        '부실판단',
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기'
    ]
]

# 비상장 피처
features = unlisted.drop(
    labels=[
        '회사명',
        '거래소코드',
        '회계년도',        
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'        
    ],
    axis=1,    
)

# 1현금흐름 4개
cash_ft1 = unlisted[
    [
        'CASH FLOW 대 부채비율',
        'CASH FLOW 대 차입금비율',
        'CASH FLOW 대 총자본비율',
        'CASH FLOW 대 매출액비율',
        '부실판단'
    ]
]

# 2안정성 9개
stability_ft1 = unlisted[
    [
        '유동비율',
        '당좌비율',
        '부채비율',
        '유동부채비율',
        '차입금의존도',
        '순운전자본비율',
        '현금비율',
        '유동자산구성비율',
        '자기자본구성비율',
        '부실판단'
    ]
]

# 3수익성 8개
profitability_ft1 = unlisted[
    [
        '자기자본순이익률',
        '경영자본순이익률',
        '매출액순이익률',
        '총자본사업이익률',
        '총자본영업이익률',
        '매출액영업이익률',
        '금융비용부담률',
        'CROE',
        '부실판단'
    ]
]

# 4성장성 6개
growth_ft1 = unlisted[
    [
        '총자산증가율',
        '총자본증가율',
        '유동자산증가율',
        '비유동자산증가율',
        '매출액증가율',
        '유형자산증가율',
        '부실판단'
    ]
]

# 5생산성 6개
productivity_ft1 = unlisted[
    [
        '부가가치',
        '총자본투자효율',
        '설비투자효율',
        '부가가치율',
        '이윤분배율',
        '자본분배율',
        '부실판단'
    ]
]

# 6활동성 8개
activity_ft1 = unlisted[
    [
        '총자산회전율',
        '매출채권회전률',
        '당좌자산회전률',
        '유형자산회전율',
        '경영자본회전률',
        '총자본회전률',
        '비유동자산회전률',
        '유동자산회전률',
        '부실판단'
    ]
]

# 7기타
etc_ft1 = unlisted[
    [
        '영업년수',
        '쭈피처',
        '영업투자현금비율',
        '영업재무현금비율', 
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'
    ]
]

In [299]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업투자현금비율', '영업재무현금비율', '도입기',
       '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [300]:
# 1현금흐름 4가지 cash
cash_ft1.describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,부실판단
count,14844.000,14844.000,14844.000,14844.000,14844.000
mean,17.069,3607.268,3.299,-20.537,0.368
std,1252.761,245521.865,23.459,457.262,0.482
min,-86467.650,-940370.860,-277.190,-21110.050,0.000
25%,-4.412,-5.200,-2.880,-4.690,0.000
50%,2.860,1.570,1.830,2.380,0.000
75%,14.400,17.475,7.360,10.400,1.000
max,113999.840,27854555.600,1108.560,21579.540,1.000


In [301]:
# 2안정성 9가지 stability
stability_ft1.describe()

,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,부실판단
count,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000
mean,396.521,323.104,1369.995,966.185,41.329,2.501,91.008,45.677,38.901,0.368
std,5278.556,5155.300,25422.528,21814.645,29.801,41.605,2442.869,28.582,25.864,0.482
min,0.070,0.000,0.000,0.000,0.000,-1830.280,-3.490,0.010,0.000,0.000
25%,56.040,31.355,62.712,38.343,17.557,-19.683,1.610,22.435,18.517,0.000
50%,107.865,71.805,164.550,105.935,41.830,2.870,8.490,43.350,35.265,0.000
75%,197.290,148.572,373.355,243.058,61.170,26.730,31.977,67.857,57.142,1.000
max,447760.110,447760.110,2238929.900,1793120.750,1109.290,99.490,275285.960,100.000,99.980,1.000


In [302]:
# 3수익성 8가지 profitability
productivity_ft1.describe()

,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000
mean,6759.214,30.727,409857.795,40.657,-83.077,10.913,0.368
std,355672.205,1686.359,16668086.473,642.157,2230.082,1040.130,0.482
min,-68167.780,-229.460,-123276675.000,-3280.110,-184890.760,-94627.640,0.000
25%,968.762,4.978,9.598,8.030,0.000,23.727,0.000
50%,2295.045,11.150,29.095,16.025,11.590,56.000,0.000
75%,4868.297,19.320,98.303,32.135,35.662,74.763,1.000
max,43327598.520,205392.780,1618245220.000,70947.820,8226.360,125.590,1.000


In [303]:
# 4성장성 6개 growth
growth_ft1.describe()

,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부실판단
count,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000
mean,6.156,6.872,42.738,268.213,173.467,196133.771,0.368
std,53.746,55.425,911.386,10684.049,6769.542,21236977.635,0.482
min,-100.000,-97.670,-99.900,-100.000,-99.910,-100.000,0.000
25%,-4.314,-5.285,-13.812,-5.080,-12.110,-5.600,0.000
50%,0.673,1.975,4.195,-0.090,3.750,-1.080,0.000
75%,11.303,12.935,25.355,12.610,22.273,8.082,1.000
max,5081.217,5081.220,97496.850,1154781.210,720869.030,2580453089.000,1.000


In [304]:
# 5생산성 6가지 productivity
productivity_ft1.describe()

,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000
mean,6759.214,30.727,409857.795,40.657,-83.077,10.913,0.368
std,355672.205,1686.359,16668086.473,642.157,2230.082,1040.130,0.482
min,-68167.780,-229.460,-123276675.000,-3280.110,-184890.760,-94627.640,0.000
25%,968.762,4.978,9.598,8.030,0.000,23.727,0.000
50%,2295.045,11.150,29.095,16.025,11.590,56.000,0.000
75%,4868.297,19.320,98.303,32.135,35.662,74.763,1.000
max,43327598.520,205392.780,1618245220.000,70947.820,8226.360,125.590,1.000


In [305]:
# 6활동성 8가지 activity
activity_ft1.describe()

,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,부실판단
count,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000,14844.000
mean,0.942,339.045,4.323,8337.509,1.131,1.013,29.415,2.852,0.368
std,1.285,30988.135,7.827,369881.882,1.387,1.268,859.925,5.168,0.482
min,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.181,4.067,1.510,0.690,0.350,0.300,0.510,1.100,0.000
50%,0.670,7.240,2.950,2.090,0.830,0.750,1.480,2.010,0.000
75%,1.249,13.550,4.973,7.123,1.450,1.310,3.800,3.350,1.000
max,36.583,3767342.720,344.060,36941363.190,43.960,43.960,93448.220,344.060,1.000


In [306]:
# 7기타 etc
etc_ft1.describe()

,영업년수,쭈피처,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,14844.000,14844.000,14844.000,14639.000,14844.000,14844.000,14844.000,14844.000,14844.000
mean,21.418,0.443,-160.160,-141.390,0.223,0.363,0.196,0.218,0.368
std,11.227,1.638,13636.282,16348.832,0.416,0.481,0.397,0.413,0.482
min,0.000,0.000,-1368867.400,-1947057.000,0.000,0.000,0.000,0.000,0.000
25%,13.000,0.085,-2.497,-1.756,0.000,0.000,0.000,0.000,0.000
50%,20.000,0.161,-0.779,-0.759,0.000,0.000,0.000,0.000,0.000
75%,27.000,0.385,-0.057,0.222,0.000,1.000,0.000,0.000,1.000
max,113.000,85.287,188183.733,227051.800,1.000,1.000,1.000,1.000,1.000


# 기업수명주기와 부실별

In [307]:
# 도입정상
도입정상 = unlisted[(unlisted['도입기'] == 1) & (unlisted['부실판단'] == 0)]

In [308]:
도입정상.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000,1298.000
mean,72536.189,2021.223,1.203,14.201,-0.844,5.016,331.262,169.157,765.463,495.488,46.307,4.656,30.530,50.254,35.115,-0.616,-0.104,-9.976,1.828,1.532,-5.493,6.624,0.030,19.474,21.779,37.295,144.552,579.145,10504.928,3189.674,12.316,338976.531,15.808,-71.783,5.637,1.095,39.148,4.588,12004.573,1.375,1.254,17.966,2.631,20.385,0.296,268.722,-2.871,1.000,0.000,0.000,0.000,0.000
std,22772.115,1.844,50.686,685.520,12.005,317.525,2869.537,1127.166,3701.462,2393.915,21.669,31.932,162.568,25.899,20.597,195.582,14.372,93.568,9.107,9.107,65.132,45.296,4.397,145.714,146.271,384.968,2131.294,20010.881,373434.691,4707.509,26.924,8200216.965,111.954,1375.774,761.062,1.898,271.818,7.511,306711.668,2.190,2.125,264.896,3.046,10.676,0.534,6379.379,26.050,0.000,0.000,0.000,0.000,0.000
min,10341.000,2014.000,-532.810,-5173.020,-89.480,-1871.620,0.310,0.240,1.750,0.120,0.000,-92.500,0.000,0.180,0.110,-4078.270,-168.970,-1799.960,-103.590,-103.710,-1729.150,-191.460,-30.868,-86.291,-86.290,-97.110,-98.930,-99.870,-99.930,-13273.590,-131.820,-17320027.900,-1467.540,-48498.650,-26993.730,0.000,0.000,0.010,0.000,0.010,0.010,0.010,0.010,0.000,0.000,-1.907,-663.394,1.000,0.000,0.000,0.000,0.000
25%,59648.000,2022.000,-6.558,-9.282,-4.180,-5.527,69.175,34.045,110.795,72.470,30.905,-14.133,1.450,30.553,20.457,-7.438,-2.785,-4.008,-0.990,-1.172,-1.897,0.390,-0.213,0.000,0.220,-6.700,-1.647,-17.312,-2.827,1037.688,4.640,9.803,6.123,0.000,16.082,0.292,4.123,1.700,1.040,0.510,0.440,0.840,1.230,12.000,0.075,0.325,-1.027,1.000,0.000,0.000,0.000,0.000
50%,77409.000,2022.000,-1.500,-1.980,-0.935,-0.950,109.075,64.645,211.505,137.925,47.710,4.080,6.530,49.175,32.100,2.815,0.960,0.900,2.545,2.335,2.290,1.245,-0.043,6.403,8.830,11.455,3.500,2.820,0.140,2256.345,9.455,26.570,11.635,8.245,48.915,0.745,6.950,3.260,2.650,0.990,0.850,1.885,2.020,19.000,0.142,0.989,-0.641,1.000,0.000,0.000,0.000,0.000
75%,91401.250,2022.000,2.685,3.775,1.710,1.695,166.240,112.355,388.722,275.783,62.550,23.343,17.695,69.850,47.440,10.262,3.730,3.397,5.307,4.970,5.938,3.087,0.000,22.531,25.090,35.153,22.422,21.455,20.360,4216.868,14.995,70.947,20.240,26.120,67.407,1.337,12.050,5.248,7.527,1.620,1.500,4.473,3.230,26.000,0.295,3.857,-0.279,1.000,0.000,0.000,0.000,0.000
max,101361.000,2022.000,1107.930,23164.090,144.470,7522.160,83360.650,36572.320,92834.240,47766.780,99.070,98.140,2704.610,100.000,98.280,5412.140,242.190,1929.130,66.050,66.040,186.030,1436.010,127.589,5081.217,5081.220,13329.590,66555.730,720869.030,13454040.000,61774.370,480.860,211718878.000,3460.750,96.710,113.560,36.583,4723.230,180.500,9011654.830,43.960,43.960,9261.970,59.280,77.000,9.262,188183.733,4.057,1.000,0.000,0.000,0.000,0.000


In [309]:
# 도입정상 결측치확인
도입정상.isna().sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [310]:
# 도입부실
도입부실 = unlisted[(unlisted['도입기'] == 1) & (unlisted['부실판단'] == 1)]

In [311]:
도입부실.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000,2010.000
mean,66228.857,2017.985,-13.374,-32.761,-7.917,-136.724,202.074,127.552,2288.387,1335.805,57.840,-6.611,33.118,43.978,25.999,18.459,-6.983,-61.203,-3.454,-3.759,-49.627,22.295,0.458,18.765,20.941,69.443,676.329,450.704,1293993.451,1720.012,6.895,347279.924,10.192,-222.264,-50.879,0.644,112.391,4.615,15380.103,0.831,0.730,84.089,2.651,19.867,0.797,119.431,-1.370,1.000,0.000,0.000,0.000,1.000
std,23488.133,2.805,33.008,214.603,11.365,938.498,1035.047,808.937,19614.241,15845.583,29.405,41.511,582.602,31.496,22.033,1830.195,31.858,336.261,15.969,15.992,271.045,92.400,20.483,49.878,52.339,413.222,11893.117,7295.498,57557190.022,5212.497,51.374,10835802.507,240.768,4180.180,2142.403,0.977,1581.214,11.256,427921.774,1.131,0.969,2133.336,6.561,11.394,2.644,2378.693,16.336,0.000,0.000,0.000,0.000,0.000
min,10062.000,2014.000,-632.290,-6513.500,-176.290,-21110.050,0.070,0.000,0.000,0.000,0.000,-364.500,-3.490,0.010,0.000,-1853.830,-949.400,-6942.270,-309.770,-309.800,-5555.060,-1554.580,-206.365,-76.407,-76.410,-98.700,-97.310,-99.830,-100.000,-52877.960,-120.130,-30708071.430,-3280.110,-184890.760,-94627.640,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-159.772,-406.853,1.000,0.000,0.000,0.000,1.000
25%,51932.250,2015.000,-14.297,-20.738,-10.168,-44.142,40.660,17.718,111.420,56.535,39.865,-26.523,0.552,15.780,7.100,-31.205,-8.628,-34.030,-5.100,-5.300,-19.920,1.103,-0.299,-0.312,-0.913,-7.550,-2.795,-15.342,-4.098,215.338,0.880,1.373,3.210,-76.095,0.000,0.059,3.302,1.162,0.223,0.130,0.110,0.173,0.660,11.000,0.111,0.411,-0.941,1.000,0.000,0.000,0.000,1.000
50%,70248.500,2018.000,-6.295,-8.545,-4.545,-10.770,87.880,43.720,260.835,155.280,58.025,-4.840,2.570,40.690,22.325,-3.475,-2.730,-6.315,-0.595,-0.900,-2.540,3.420,-0.023,4.853,6.865,11.930,1.625,2.280,-0.480,1141.800,4.880,12.170,11.580,0.000,33.955,0.370,6.470,2.645,1.440,0.550,0.470,1.020,1.550,18.000,0.235,1.637,-0.628,1.000,0.000,0.000,0.000,1.000
75%,84738.250,2020.000,-2.330,-2.762,-1.640,-2.947,142.502,89.483,697.200,421.660,75.553,16.610,9.755,69.338,39.047,2.700,0.925,1.420,2.380,2.058,3.945,15.027,0.041,21.529,25.602,42.183,21.700,36.282,14.975,2574.528,10.527,56.898,28.328,9.905,67.175,0.889,14.342,4.918,7.138,1.120,0.988,3.325,2.790,25.000,0.676,6.637,-0.286,1.000,0.000,0.000,0.000,1.000
max,101548.000,2022.000,190.180,1879.210,32.100,2636.780,28008.190,26092.520,664184.670,664151.380,405.580,98.640,25930.710,100.000,99.670,81898.380,286.040,3177.900,50.010,50.000,98.960,1740.040,818.231,1130.384,1130.380,8829.820,418389.230,241171.310,2580453089.000,75094.030,2179.890,402503633.330,4384.980,2781.640,125.590,23.652,59829.560,246.350,17320295.670,24.120,20.200,93448.220,151.080,74.000,56.551,83902.590,469.354,1.000,0.000,0.000,0.000,1.000


In [312]:
# 도입부실 결측치확인
도입부실.isna().sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [313]:
# 성장정상
성장정상 = unlisted[(unlisted['성장기'] == 1) & (unlisted['부실판단'] == 0)]

In [314]:
# 결측치확인
성장정상.isna().sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [315]:
# 성장부실
성장부실 = unlisted[(unlisted['성장기'] == 1) & (unlisted['부실판단'] == 1)]

In [316]:
# 결측치 확인
성장부실.isna().sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [317]:
# inf 확인
성장부실.isin([np.inf]).sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [318]:
# inf 값 대체
inf_col = 성장부실.isin([np.inf]).sum().sort_values(ascending=False).index[0]
max_value = 성장부실[inf_col].replace(np.inf, np.nan).max()
성장부실[inf_col] = 성장부실[inf_col].replace(np.inf, max_value)

In [319]:
성장부실[inf_col].isin([np.inf]).sum()
# .sort_values(ascending=False)

0

In [320]:
# 성숙정상
성숙정상 = unlisted[(unlisted['성숙기'] == 1) & (unlisted['부실판단'] == 0)]

In [321]:
# 성숙정상 결측치 확인
성숙정상.isna().sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [322]:
# 성숙부실
성숙부실 = unlisted[(unlisted['성숙기'] == 1) & (unlisted['부실판단'] == 1)]

In [323]:
# 성숙부실 결측치 확인
성숙부실.isna().sum().sort_values(ascending=False)

회사명                  0
비유동자산증가율             0
유형자산증가율              0
부가가치                 0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
매출액증가율               0
유동자산증가율              0
거래소코드                0
총자본증가율               0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율             0
자기자본구성비율             0
자기자본순이익률   

In [324]:
# 정규성 확인 하기
na_col0 = 성숙부실.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 성숙부실[na_col0].dropna()

In [325]:
# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data0)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [326]:
# # 안더슨-달링 검정 수행
# result = anderson(data0)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")

In [327]:
# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data0, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [328]:
# # fillna median
# 성숙부실[na_col0].fillna(data0.median(), inplace=True)

In [329]:
# 쇠퇴정상
쇠퇴정상 = unlisted[(unlisted['쇠퇴기'] == 1) & (unlisted['부실판단'] == 0)]

In [330]:
# 결측치 확인
쇠퇴정상.isna().sum().sort_values(ascending=False)

영업재무현금비율             94
회사명                   0
유형자산회전율               0
유형자산증가율               0
부가가치                  0
총자본투자효율               0
설비투자효율                0
부가가치율                 0
이윤분배율                 0
자본분배율                 0
총자산회전율                0
매출채권회전률               0
당좌자산회전률               0
경영자본회전률               0
비유동자산증가율              0
총자본회전률                0
비유동자산회전률              0
유동자산회전률               0
영업년수                  0
쭈피처                   0
영업투자현금비율              0
도입기                   0
성숙기                   0
성장기                   0
쇠퇴기                   0
매출액증가율                0
유동자산증가율               0
거래소코드                 0
총자본증가율                0
회계년도                  0
CASH FLOW 대 부채비율      0
CASH FLOW 대 차입금비율     0
CASH FLOW 대 총자본비율     0
CASH FLOW 대 매출액비율     0
유동비율                  0
당좌비율                  0
부채비율                  0
유동부채비율                0
차입금의존도                0
순운전자본비율               0
현금비율                  0
유동자산구성비율        

In [331]:
# 정규성 확인하기
na_col0 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴정상[na_col0].dropna()

In [332]:
# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.015714287757873535, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [333]:
# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 486.34470614196334
유의수준 15.0%: 임계값 = 0.574
유의수준 10.0%: 임계값 = 0.654
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.915
유의수준 1.0%: 임계값 = 1.089
데이터는 정규 분포를 따르지 않습니다.


In [334]:
# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.16583780243677781, p-value: 9.700955661708916e-32
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [335]:
# 쇠퇴정상 fillna median
쇠퇴정상[na_col0].fillna(data0.median(), inplace=True)

In [336]:
# 쇠퇴부실
쇠퇴부실 = unlisted[(unlisted['쇠퇴기'] == 1) & (unlisted['부실판단'] == 1)]

In [337]:
# 결측치 확인
쇠퇴부실.isna().sum().sort_values(ascending=False)

영업재무현금비율             111
회사명                    0
유형자산회전율                0
유형자산증가율                0
부가가치                   0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
경영자본회전률                0
비유동자산증가율               0
총자본회전률                 0
비유동자산회전률               0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
매출액증가율                 0
유동자산증가율                0
거래소코드                  0
총자본증가율                 0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0
순운전자본비율                0


In [338]:
# 정규성 확인하기
na_col0 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴부실[na_col0].dropna()

In [339]:
# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.026852786540985107, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [340]:
# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 635.6933266182614
유의수준 15.0%: 임계값 = 0.575
유의수준 10.0%: 임계값 = 0.654
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.089
데이터는 정규 분포를 따르지 않습니다.


In [341]:
# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.2163207316705122, p-value: 1.0374239803111806e-71
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [342]:
# fillna(median)
쇠퇴부실[na_col0].fillna(data0.median(), inplace=True)

# to_csv

In [343]:
unlisted.loc[도입부실.index] = 도입부실
unlisted.loc[도입정상.index] = 도입정상
unlisted.loc[성장부실.index] = 성장부실
unlisted.loc[성장정상.index] = 성장정상
unlisted.loc[성숙부실.index] = 성숙부실
unlisted.loc[성숙정상.index] = 성숙정상
unlisted.loc[쇠퇴부실.index] = 쇠퇴부실
unlisted.loc[쇠퇴정상.index] = 쇠퇴정상

In [344]:
unlisted.to_csv('../1.데이터셋/Train_Test/unlisted_filled_train_data.csv', index=False)

In [345]:
# filled_train_data = pd.concat([도입부실, 도입정상, 성장부실, 성장정상, 성숙부실, 성숙정상, 쇠퇴부실, 쇠퇴정상], axis=0, ignore_index=True)

In [346]:
# filled_train_data.columns

In [347]:
# len(filled_train_data)

In [348]:
# filled_train_data.to_csv('../data/datasets/unlisted_filled_train_data.csv', index=False)